In [1]:
import os
import dataflows as DF
import dotenv
import dataflows_airtable as DFA

dotenv.load_dotenv()

URL=os.environ['URL']
AIRTABLE_BASE=os.environ['AIRTABLE_BASE']
AIRTABLE_TABLE=os.environ['AIRTABLE_TABLE']
AIRTABLE_API_KEY=os.environ['AIRTABLE_API_KEY']

In [3]:
def unwind(base):
    def func(rows):
        for row in rows:
            out_ = dict((k, v) for k, v in row.items() if k in base)
            for k, v in row.items():
                if k in base or k in ['indicator', 'value', 'estimated']:
                    continue
                out = dict(
                    **out_,
                    indicator=k,
                    value=0.5 if v is None else v,
                    estimated=v is None
                )
                yield out
            
    return DF.Flow(
        DF.add_field('indicator', 'string'),
        DF.add_field('value', 'number'),
        DF.add_field('estimated', 'boolean'),
        func,
        DF.select_fields(['country', 'data_type', 'indicator', 'value'])
    )


DF.Flow(
    DF.load(URL),
    DF.update_resource(-1, name='data'),
    DF.rename_fields({
        'מדינה': 'country',
        'data type': 'data_type',
    }),
    unwind(['country', 'data_type']),
    DF.printer(tablefmt='html'),
    DFA.dump_to_airtable({
        (AIRTABLE_BASE, AIRTABLE_TABLE): {
            'resource-name': 'data',
            'typecast': True
        }
    }, apikey=AIRTABLE_API_KEY)
).process()

#,country (string),data_type (string),indicator (string),value (number)
1,luxembourg,raw,פנאי,15.20
2,luxembourg,raw,ימי חופשה לעובדים,37.00
3,luxembourg,raw,זמן בדרכים,29.00
4,luxembourg,raw,שעות שבועיות למשרה מלאה,40.36
5,luxembourg,raw,שביעות רצון מחינוך,
6,luxembourg,raw,הפוגת אבות (סך שבועות משולמים היטב),27.80
7,luxembourg,raw,הפוגת אמהות (סך שבועות משולמים היטב),37.54
8,luxembourg,raw,פער כניסה למסגרות,21.00
9,luxembourg,raw,הכנסה פנויה למשפחה עם ילדים,
10,luxembourg,raw,שיעור הטבות וקצבאות למשפחה מהכנסה,0.14


(<datapackage.package.Package at 0x12b2c3520>, {})